In [52]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("US_youtube_trending_data.csv")

columns_to_delet = ['video_id', 'title', 'channelId', 'channelTitle', 'tags', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'description']

for column in columns_to_delet : 
    df.drop(column, axis='columns', inplace=True)

df['days_uploaded'] = df.apply(lambda row: (datetime.fromisoformat(row['trending_date'].split('T')[0] + "T00:00:00") - datetime.fromisoformat(row['publishedAt'].split('T')[0] + "T00:00:00")).days, axis=1)

df.drop('publishedAt', axis='columns', inplace=True)
df.drop('trending_date', axis='columns', inplace=True)

df.to_csv('exctracted_data.csv', index=False)

,categoryId,view_count,likes,dislikes,comment_count,days_uploaded
0,22,1514614,156908,5855,35313,1
1,20,2381688,146739,2794,16549,1
2,24,2038853,353787,2628,40221,1
3,10,496771,23251,1856,7647,1
4,26,1123889,45802,964,2196,1
...,...,...,...,...,...,...
245982,23,563422,68917,0,1822,7
245983,20,2440589,114121,0,5828,7
245984,17,474333,5730,0,343,6
245985,24,804338,29540,0,1972,6
